In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

: 

In [3]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_DIR = "../src/data/"

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    f"{DATASET_DIR}/train", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")

val_generator = datagen.flow_from_directory(
    f"{DATASET_DIR}/val", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")

test_generator = datagen.flow_from_directory(
    f"{DATASET_DIR}/test", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")


Found 4359 images belonging to 5 classes.
Found 932 images belonging to 5 classes.
Found 939 images belonging to 5 classes.


In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # On gèle les couches de ResNet

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax')(x)  # 5 classes

model = Model(inputs=base_model.input, outputs=predictions)


94765736/94765736 [==============================] - 97s 1us/step


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


Epoch 1/10
137/137 [==============================] - 908s 7s/step - loss: 1.6935 - accuracy: 0.2684 - val_loss: 1.4317 - val_accuracy: 0.3476
Epoch 2/10
137/137 [==============================] - 807s 6s/step - loss: 1.5649 - accuracy: 0.3051 - val_loss: 1.4208 - val_accuracy: 0.3466
Epoch 3/10
137/137 [==============================] - 798s 6s/step - loss: 1.5239 - accuracy: 0.3230 - val_loss: 1.4201 - val_accuracy: 0.3455
Epoch 4/10
137/137 [==============================] - 909s 7s/step - loss: 1.5042 - accuracy: 0.3063 - val_loss: 1.4167 - val_accuracy: 0.3487
Epoch 5/10
137/137 [==============================] - ETA: 0s - loss: 1.4785 - accuracy: 0.3386

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Accuracy sur test : {test_acc:.2f}")